In [9]:
import os
import sys

#sys.path.append("/home/me/mypy")

import scipy.io as sio

#import matplotlib
import matplotlib

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
import pylab 
import matplotlib.cm as cm 

import random

import lmdb
import caffe

def convert_dataset(image_filename, mat_ground_truth, db_filename):
    
    
    # load mat file containing groundtruth 
    GROUND_TRUTH_PATH = os.path.expanduser(mat_ground_truth)
    LMDB_PATH = db_filename

    #get ground truth data
    #temp = sio.loadmat(GROUND_TRUTH_PATH)
    gt_data = sio.loadmat(GROUND_TRUTH_PATH)['truth']
    
    shp = gt_data.shape
    rows = shp[0]
    
    #loop through all rows in matrix
    idx = 1
    n_tp = 2
    n_fp = 2
    n_else = 2
    
    map_size = 480*640*3*2*3000
    env = lmdb.open(LMDB_PATH, map_size=map_size)
    
    for row in gt_data:
    
    
        #find two TP
        tp_indices = np.where(row == 1) 
        
        if (tp_indices[0]).size != 0 :

            #find two else
            else_indices = np.where(row == 0)

            #find two FP
            fp_indices = np.where(row == -1)
            
            
            name1 = '%04d' % (idx)
            
            # get n_tp TP's
            if (tp_indices[0]).size !=0 :
                random.shuffle(tp_indices[0])
            
                
            #name2 = '%04d' % (tp_indices[0])[0]
            
            # get n_else else's
            if (else_indices[0]).size !=0 :
                random.shuffle(else_indices[0])
            
            #else_choice = random.randint(0,len((else_indices[0]))-1)
            #name3 = '%04d' % (else_indices[0])[choice]
            
            
            # get n_fp FP's
            if (fp_indices[0]).size !=0 :
                random.shuffle(fp_indices[0])
                
                
            # display images    
            fname1 = image_filename + name1 + '.jpg'

            image=Image.open(fname1).convert("L")
            arr=np.asarray(image)
            
            query_img = caffe.io.load_image(fname1)
            query_img = np.transpose(query_img, (2,0,1))
            print query_img.shape
            print type(query_img)
            #query_img2 = np.asarray(Image.open(fname1))
            #query_img3 = np.concatenate((query_img,query_img2),axis=2)
            #print(query_img.shape)
            #print(query_img3.shape)
            #f.add_subplot(3, 1, n+1)  # this line outputs images on top of each other
            # f.add_subplot(1, 2, n)  # this line outputs images side-by-side
            #pylab.imshow(arr,cmap=cm.Greys_r)
            #pylab.title('query image')
            #pylab.show()
            
            
            for i in xrange(0,min([n_tp,len(tp_indices[0])])):
                name = '%04d' % ((tp_indices[0])[i] + 1)
                fname = image_filename + name + '.jpg'
                
                image=Image.open(fname).convert("L")
                arr=np.asarray(image)
                tp_img = caffe.io.load_image(fname)
                tp_img = np.transpose(tp_img, (2,0,1))
                joint_img = np.concatenate((query_img,tp_img), axis=0)
                #f.add_subplot(3, 1, n+1)  # this line outputs images on top of each other
                #f.add_subplot(1, 2, n)  # this line outputs images side-by-side
                #pylab.imshow(arr,cmap=cm.Greys_r)
                #pylab.title('true positive')
                #pylab.show()
                print([idx -1,i])
                print(gt_data[idx-1,(tp_indices[0])[i]])
                
              
                
                with env.begin(write=True) as txn:
                    print joint_img.shape
                    print joint_img.nbytes
                    #datum = caffe.proto.caffe_pb2.Datum()
                    #datum.channels = joint_img.shape[2]
                    #datum.height = joint_img.shape[0]
                    #datum.width = joint_img.shape[1]
                    #datum.float_data = joint_img.tobytes()
                    #datum.label = int(1.0)
                    joint_img = joint_img.astype(np.float32)
                    datum = caffe.io.array_to_datum(joint_img,int(1.0))
                    txn_id = int(str(idx -1) + str((tp_indices[0])[i]))
                    str_id = '{:08}'.format(txn_id)
                    
                    txn.put(str_id.encode('ascii'), datum.SerializeToString())
                #txn.commit()
                    
            
                    
                
                
                
            if (else_indices[0]).size !=0 :    
                for i in xrange(0,min([n_else,len(else_indices[0])])):
                    name = '%04d' % ((else_indices[0])[i] + 1)
                    fname = image_filename + name + '.jpg'

                    image=Image.open(fname).convert("L")
                    arr=np.asarray(image)
                    #f.add_subplot(3, 1, n+1)  # this line outputs images on top of each other
                    # f.add_subplot(1, 2, n)  # this line outputs images side-by-side
                    #pylab.imshow(arr,cmap=cm.Greys_r)
                    #pylab.title('not related')
                    #pylab.show()
                    print(gt_data[idx-1,(else_indices[0])[i]])
                    
            if (fp_indices[0]).size !=0 :    
                for i in xrange(0,min([n_fp,len(fp_indices[0])])):
                    name = '%04d' % ((fp_indices[0])[i] + 1)
                    fname = image_filename + name + '.jpg'

                    image=Image.open(fname).convert("L")
                    arr=np.asarray(image)
                    #f.add_subplot(3, 1, n+1)  # this line outputs images on top of each other
                    # f.add_subplot(1, 2, n)  # this line outputs images side-by-side
                    #pylab.imshow(arr,cmap=cm.Greys_r)
                    #pylab.title('false positive')
                    #pylab.show()
                    print(gt_data[idx-1,(fp_indices[0])[i]])
                    
                    
                    
            
            #name1 = '/home/sastrygrp/code/loop_closures/data/Lip6IndoorDataSet/Images/' + name1 +'.jpg'
            #name2 = '/home/sastrygrp/code/loop_closures/data/Lip6IndoorDataSet/Images/' + name2 +'.jpg'
            #name3 = '/home/sastrygrp/code/loop_closures/data/Lip6IndoorDataSet/Images/' + name3 +'.jpg'


            #f = pylab.figure()
            #for n, fname in enumerate((name1, name2,name3)):
                #image=Image.open(fname).convert("L")
                #arr=np.asarray(image)
                #f.add_subplot(3, 1, n+1)  # this line outputs images on top of each other
                # f.add_subplot(1, 2, n)  # this line outputs images side-by-side
                #pylab.imshow(arr,cmap=cm.Greys_r)
            #pylab.title('Double image')
            #pylab.show()
        
            #print(tp_indices)
            #print(else_indices)
            print(idx)
            #raw_input("Press Enter to continue...")
            
            
            
        idx = idx+1
        
    
    #env.close()
    
img_filename = '/home/sastrygrp/code/loop_closures/data/Images/'
grnd_truth_filename ='/home/sastrygrp/code/loop_closures/data/NewCollegeGroundTruth.mat'
db_filename = '/home/sastrygrp/code/loop_closures/data/NewCollege'

convert_dataset(img_filename, grnd_truth_filename, db_filename)
print 'done writing'

(3, 480, 640)
<type 'numpy.ndarray'>
[188, 0]
1
(6, 480, 640)
7372800
[188, 1]
1
(6, 480, 640)
7372800
0
0
189
(3, 480, 640)
<type 'numpy.ndarray'>
[189, 0]
1
(6, 480, 640)
7372800
[189, 1]
1
(6, 480, 640)
7372800
0
0
190
(3, 480, 640)
<type 'numpy.ndarray'>
[190, 0]
1
(6, 480, 640)
7372800
[190, 1]
1
(6, 480, 640)
7372800
0
0
191
(3, 480, 640)
<type 'numpy.ndarray'>
[191, 0]
1
(6, 480, 640)
7372800
[191, 1]
1
(6, 480, 640)
7372800
0
0
192
(3, 480, 640)
<type 'numpy.ndarray'>
[192, 0]
1
(6, 480, 640)
7372800
[192, 1]
1
(6, 480, 640)
7372800
0
0
193
(3, 480, 640)
<type 'numpy.ndarray'>
[193, 0]
1
(6, 480, 640)
7372800
[193, 1]
1
(6, 480, 640)
7372800
0
0
194
(3, 480, 640)
<type 'numpy.ndarray'>
[194, 0]
1
(6, 480, 640)
7372800
[194, 1]
1
(6, 480, 640)
7372800
0
0
195
(3, 480, 640)
<type 'numpy.ndarray'>
[195, 0]
1
(6, 480, 640)
7372800
[195, 1]
1
(6, 480, 640)
7372800
0
0
196
(3, 480, 640)
<type 'numpy.ndarray'>
[196, 0]
1
(6, 480, 640)
7372800
[196, 1]
1
(6, 480, 640)
7372800
0
0
197
(

MapFullError: mdb_put: MDB_MAP_FULL: Environment mapsize limit reached